In [1]:
import numpy as np
import os
import pandas as pd
import re
from plotfx import *
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FuncFormatter

%matplotlib inline

## High Community Transmission

In [3]:
outfile = 'allbeta_noNPI_vax_hicom'

In [4]:
df = pickle.load(open(outfile + ".pkl", "rb" ))
df.head()

,rep,p_vax,tstep,ext_inf_ct,test_freq,ia_nodes,is_nodes,test_pos_schd,test_pos_symp,q_schd,...,cum_list_inf,n_cum_inf,all_test,n_test_pos,cum_list_test,n_cum_pos,all_quar,n_quar,cum_list_quar,n_cum_quar
0,1,0.0,0,128,0,[],[684],[],[],[],...,{684},1,[],0,{},0,[],0,{},0
1,1,0.0,1,128,0,[],[684],[],[],[],...,{684},1,[],0,{},0,[],0,{},0
2,1,0.0,2,128,0,"[690, 159]",[684],[],[],[],...,"{690, 684, 159}",3,[],0,{},0,[],0,{},0
3,1,0.0,3,128,0,"[690, 159]","[684, 661]",[],[],[],...,"{690, 684, 661, 159}",4,[],0,{},0,[],0,{},0
4,1,0.0,4,128,0,[159],"[684, 661]",[],[684],[],...,"{690, 684, 661, 159}",4,[684],1,{684},1,[],0,{},0


### Plot Infections Over Time for Multiple Immunity Levels

In [ ]:
annotate = ["0%\nImmune", "20%\nImmune", "40%\nImmune", "60%\nImmune", "80%\nImmune"]
title = 'Every {freq} Days'
titles = [title.format(freq = f) for f in [3,7,14,28]] + ['Symptomatic Only']
titles

In [ ]:
# Save the number needed to hit 10% of student population infected
n10 = 0.1*692

#### R0 approx. 1.5

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==3) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_inf'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_inf'].mean().index, pltdf.groupby('tstep')['n_inf'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if t10<=112:
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 36, 70, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(175, 40, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Number Infectious', fontsize = 20)
        if j==4:
            axs2 = axs[i,j].twinx()
            axs2.yaxis.set_ticks(np.arange(0,1,0.2))
            axs2.yaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))
            axs2.set_ylabel('Percent Infectious', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        axs2.tick_params(axis = 'y', labelsize = 12)
        
plt.savefig('InfTime_R15_Allvax_hicom.eps', bbox_inches='tight', transparent=True);
#plt.savefig('InfTime_R15_Allvax_hicom.jpg', bbox_inches='tight', transparent=True);

#### R0 approx. 3.0

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==6) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_inf'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_inf'].mean().index, pltdf.groupby('tstep')['n_inf'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if (t10<=112) & (t10<=70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 + 6, 120, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
        else:
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 35, 120, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(175, 60, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Number Infectious', fontsize = 20)
        if j==4:
            axs2 = axs[i,j].twinx()
            axs2.yaxis.set_ticks(np.arange(0,1,0.2))
            axs2.yaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))
            axs2.set_ylabel('Percent Infectious', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        axs2.tick_params(axis = 'y', labelsize = 12)
        
#plt.savefig('InfTime_R30_Allvax_hicom.eps', bbox_inches='tight', transparent=True);
plt.savefig('InfTime_R30_Allvax_hicom.jpg', bbox_inches='tight', transparent=True);

#### R0 approx. 4.5

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==9) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_inf'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_inf'].mean().index, pltdf.groupby('tstep')['n_inf'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if t10<=112:
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 + 7, 175, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(175, 100, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Number Infectious', fontsize = 20)
        if j==4:
            axs2 = axs[i,j].twinx()
            axs2.yaxis.set_ticks(np.arange(0,1,0.2))
            axs2.yaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))
            axs2.set_ylabel('Percent Infectious', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        axs2.tick_params(axis = 'y', labelsize = 12)
        
plt.savefig('InfTime_R45_Allvax_hicom.eps', bbox_inches='tight', transparent=True);
#plt.savefig('InfTime_R45_Allvax_hicom.jpg', bbox_inches='tight', transparent=True);

### Plot Number of Positive Tests Over Time for Multiple Immunity Levels

#### R0 approx. 1.5

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==3) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_test_pos'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_test_pos'].mean().index, pltdf.groupby('tstep')['n_test_pos'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if (t10<=112) & (t10<=70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 + 7, 21, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
        else:
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 35, 21, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(140, 10, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Positive Tests', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        
plt.savefig('TestsTime_R15_Allvax_hicom.eps', bbox_inches='tight', transparent=True);
#plt.savefig('TestsTime_R15_Allvax_hicom.jpg', bbox_inches='tight', transparent=True);

#### R0 approx. 3.0

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==6) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_test_pos'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_test_pos'].mean().index, pltdf.groupby('tstep')['n_test_pos'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if (t10<=112) & (t10<=70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 + 7, 30, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
        else:
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 35, 30, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(140, 20, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Positive Tests', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        
plt.savefig('TestsTime_R30_Allvax_hicom.eps', bbox_inches='tight', transparent=True);
#plt.savefig('TestsTime_R30_Allvax_hicom.jpg', bbox_inches='tight', transparent=True);

#### R0 approx. 4.5

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==9) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_test_pos'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_test_pos'].mean().index, pltdf.groupby('tstep')['n_test_pos'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if (t10<=112) & (t10<=70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 + 7, 35, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
        else:
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 35, 35, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(140, 20, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Positive Tests', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        
plt.savefig('TestsTime_R45_Allvax_hicom.eps', bbox_inches='tight', transparent=True);
#plt.savefig('TestsTime_R45_Allvax_hicom.jpg', bbox_inches='tight', transparent=True);

### Plot Number of Isolations Over Time for Multiple Immunity Levels

#### R0 approx. 1.5

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==3) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_quar'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_quar'].mean().index, pltdf.groupby('tstep')['n_quar'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if (t10<=112) & (t10<=70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 + 7, 75, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
        else:
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 35, 75, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(140, 40, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Isolations', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        
plt.savefig('IsoTime_R15_Allvax_hicom.eps', bbox_inches='tight', transparent=True);
#plt.savefig('IsoTime_R15_Allvax_hicom.jpg', bbox_inches='tight', transparent=True);

#### R0 approx. 3.0

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==6) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_quar'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_quar'].mean().index, pltdf.groupby('tstep')['n_quar'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if (t10<=112) & (t10<=70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 + 7, 110, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
        else:
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 35, 110, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(140, 60, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Isolations', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        
#plt.savefig('IsoTime_R30_Allvax_hicom.eps', bbox_inches='tight', transparent=True);
plt.savefig('IsoTime_R30_Allvax_hicom.jpg', bbox_inches='tight', transparent=True);

#### R0 approx. 4.5

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==9) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_quar'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_quar'].mean().index, pltdf.groupby('tstep')['n_quar'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if (t10<=112) & (t10<=70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 + 7, 140, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
        else:
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 35, 140, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(140, 80, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Isolations', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        
#plt.savefig('IsoTime_R45_Allvax_hicom.eps', bbox_inches='tight', transparent=True);
plt.savefig('IsoTime_R45_Allvax_hicom.jpg', bbox_inches='tight', transparent=True);

## Low Community Transmission

In [ ]:
outfile = 'allbeta_noNPI_vax_locom'

In [ ]:
df = pickle.load(open(outfile + ".pkl", "rb" ))
df.head()

### Plot Infections Over Time for Multiple Immunity Levels

In [ ]:
annotate = ["0%\nImmune", "20%\nImmune", "40%\nImmune", "60%\nImmune", "80%\nImmune"]
title = 'Every {freq} Days'
titles = [title.format(freq = f) for f in [3,7,14,28]] + ['Symptomatic Only']
titles

In [ ]:
# Save the number needed to hit 10% of student population infected
n10 = 0.1*692

#### R0 approx. 1.5

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==3) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_inf'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_inf'].mean().index, pltdf.groupby('tstep')['n_inf'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if t10<=112:
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 37, 50, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(175, 30, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Number Infectious', fontsize = 20)
        if j==4:
            axs2 = axs[i,j].twinx()
            axs2.yaxis.set_ticks(np.arange(0,1,0.2))
            axs2.yaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))
            axs2.set_ylabel('Percent Infectious', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        axs2.tick_params(axis = 'y', labelsize = 12)
            
plt.savefig('InfTime_R15_Allvax_locom.eps', bbox_inches='tight', transparent=True);
#plt.savefig('InfTime_R15_Allvax_locom.jpg', bbox_inches='tight', transparent=True);

#### R0 approx. 3.0

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==3) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_inf'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_inf'].mean().index, pltdf.groupby('tstep')['n_inf'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if t10<=112:
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 37, 50, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(175, 30, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Number Infectious', fontsize = 20)
        if j==4:
            axs2 = axs[i,j].twinx()
            axs2.yaxis.set_ticks(np.arange(0,1,0.2))
            axs2.yaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))
            axs2.set_ylabel('Percent Infectious', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        axs2.tick_params(axis = 'y', labelsize = 12)
            
plt.savefig('InfTime_R30_Allvax_locom.eps', bbox_inches='tight', transparent=True);
#plt.savefig('InfTime_R30_Allvax_locom.jpg', bbox_inches='tight', transparent=True);

#### R0 approx. 4.5

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==3) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_inf'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_inf'].mean().index, pltdf.groupby('tstep')['n_inf'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if t10<=112:
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 37, 50, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(175, 30, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Number Infectious', fontsize = 20)
        if j==4:
            axs2 = axs[i,j].twinx()
            axs2.yaxis.set_ticks(np.arange(0,1,0.2))
            axs2.yaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))
            axs2.set_ylabel('Percent Infectious', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        axs2.tick_params(axis = 'y', labelsize = 12)
            
plt.savefig('InfTime_R45_Allvax_locom.eps', bbox_inches='tight', transparent=True);
#plt.savefig('InfTime_R45_Allvax_locom.jpg', bbox_inches='tight', transparent=True);

### Plot Number of Positive Tests Over Time for Multiple Immunity Levels

#### R0 approx. 1.5

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==3) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_test_pos'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_test_pos'].mean().index, pltdf.groupby('tstep')['n_test_pos'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if (t10<=112) & (t10<=70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 + 7, 13, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
        if (t10<=112) & (t10>70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 36, 13, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(140, 8, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Positive Tests', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        
plt.savefig('TestsTime_R15_Allvax_locom.eps', bbox_inches='tight', transparent=True);
#plt.savefig('TestsTime_R15_Allvax_locom.jpg', bbox_inches='tight', transparent=True);

#### R0 approx. 3.0

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==6) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_test_pos'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_test_pos'].mean().index, pltdf.groupby('tstep')['n_test_pos'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if (t10<=112) & (t10<=70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 + 7, 18, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
        if (t10<=112) & (t10>70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 36, 18, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(140, 10, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Positive Tests', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        
plt.savefig('TestsTime_R30_Allvax_locom.eps', bbox_inches='tight', transparent=True);
#plt.savefig('TestsTime_R30_Allvax_locom.jpg', bbox_inches='tight', transparent=True);

#### R0 approx. 4.5

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==9) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_test_pos'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_test_pos'].mean().index, pltdf.groupby('tstep')['n_test_pos'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if (t10<=112) & (t10<=70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 + 7, 28, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
        if (t10<=112) & (t10>70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 36, 28, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(140, 15, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Positive Tests', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        
plt.savefig('TestsTime_R45_Allvax_locom.eps', bbox_inches='tight', transparent=True);
#plt.savefig('TestsTime_R45_Allvax_locom.jpg', bbox_inches='tight', transparent=True);

### Plot Number of Isolations Over Time for Multiple Immunity Levels

#### R0 approx. 1.5

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==3) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_quar'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_quar'].mean().index, pltdf.groupby('tstep')['n_quar'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if (t10<=112) & (t10<=70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 + 7, 55, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
        if (t10<=112) & (t10>70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 36, 55, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(140, 30, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Isolations', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        
#plt.savefig('IsoTime_R15_Allvax_locom.eps', bbox_inches='tight', transparent=True);
plt.savefig('IsoTime_R15_Allvax_locom.jpg', bbox_inches='tight', transparent=True);

#### R0 approx. 3.0

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==6) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_quar'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_quar'].mean().index, pltdf.groupby('tstep')['n_quar'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if (t10<=112) & (t10<=70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 + 7, 80, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
        if (t10<=112) & (t10>70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 36, 80, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(140, 40, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Isolations', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        
#plt.savefig('IsoTime_R30_Allvax_locom.eps', bbox_inches='tight', transparent=True);
plt.savefig('IsoTime_R30_Allvax_locom.jpg', bbox_inches='tight', transparent=True);

#### R0 approx. 4.5

In [ ]:
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'

f, axs = plt.subplots(5, 5, figsize = (25, 25), sharex = True, sharey = True)
#f.suptitle('Testing Frequency', fontsize = 20, y = 0.93)
f.subplots_adjust(wspace=0, hspace=0)

# Loop through rows
for (i, p_vax) in enumerate(df.p_vax.unique()):
    
    # Loop through the testing values (columns)
    for (j, tf) in enumerate(df.test_freq.unique()):
        
        pltdf = df.loc[(df.beta==9) & (df.p_vax == p_vax) & (df.test_freq==tf)]
        
        if tf==0:
            j = 4
        else:
            j = j-1
        
        # Plot individual runs
        for k in range(1, pltdf['rep'].max() + 1):
            axs[i,j].plot(pltdf[pltdf.rep == k]['tstep'].values, pltdf[pltdf.rep == k]['n_quar'].values, 'lightgrey', linewidth = 0.75)
        
        # Plot average run and time to 10% infected
        axs[i,j].plot(pltdf.groupby('tstep')['n_quar'].mean().index, pltdf.groupby('tstep')['n_quar'].mean().values)
        t10 = pltdf.loc[pltdf['n_cum_inf']>=n10].groupby('rep')['tstep'].first().mean()
        if (t10<=112) & (t10<=70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 + 7, 110, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
        if (t10<=112) & (t10>70):
            axs[i,j].axvline(x = t10, color = 'darkred', linestyle = 'dashed')
            axs[i,j].text(t10 - 36, 110, str(np.round(t10)) + ' Days', color = 'darkred', fontsize = 14)
            
        # Annotate immunity levels if last column
        if (j)==(len(titles)-1):
            axs[i,j].text(140, 60, annotate[i], fontsize = 20, ha='center', va='center')
            
        # Only set title if first row
        if i==0:
            axs[i,j].set_title(titles[j], fontsize = 20)
            
        # Only set xlabel if last row
        if i==4:
            axs[i,j].set_xlabel('Time (Days)', fontsize = 20)
            
        # Only set ylabel if first column
        if j==0:
            axs[i,j].set_ylabel('Isolations', fontsize = 20)
            
        # Adjust ticks and labels
        axs[i,j].tick_params(axis = 'x', labelsize = 12)
        axs[i,j].tick_params(axis = 'y', labelsize = 12)
        
#plt.savefig('IsoTime_R45_Allvax_locom.eps', bbox_inches='tight', transparent=True);
plt.savefig('IsoTime_R45_Allvax_locom.jpg', bbox_inches='tight', transparent=True);